In [95]:
def createCInit(D, minSup):
    # 生成第一个候选序列，即长度为1的集合序列
    C1 = []
    suffixDic={}  # 集合对应的每个序列的所在位置的开始位置，{(1,):{0:2,3:1}}表示前缀(1,)在0序列的结尾为2， 3序列的结尾为1
    for i in range(len(D)):
        for idx, item in enumerate(D[i]):
            if tuple([item]) not in C1:
                suffixDic[tuple([item])]={}
                C1.append(tuple([item]))
            if i not in suffixDic[tuple([item])].keys():
                suffixDic[tuple([item])][i]=idx
    # 根据当前候选集合删除低support的候选集 得到高于支持度的集合

    L1 = C1.copy()# 必须替换一个进行copy操作
    for iset in C1:
        if (len(suffixDic[iset])/len(D)) < minSup:
            L1.remove(iset)
            suffixDic.pop(iset)
    return L1, suffixDic

In [79]:
def genNewFixAndFixDict(D, Lk, suffixDic, minSup):
    # 生成长度+1的新的候选集合
    newLk = []
    # 访问每一个前缀
    for Ck in Lk:
    #     print("\n---------------------------------------------------------------------------------------------\n")
        itemAllCount = {}  # 统计item的支持度计数
        # 访问出现前缀的每一行  
        for i in suffixDic[Ck].keys():
            # Ck是('b',)形式C
            # 从前缀开始访问每个字符
            itemThisRow = {}# 统计item每一行后缀的情况
            # suffixDic[Ck]是一个字典，i是key，suffixDic[Ck][i]是对应的后缀的位置，也是对应data里的行数
    #         print("\n")
    #         print(Ck,"到第%d行为止"%i)
            for j in range(suffixDic[Ck][i] + 1, len(D[i])):
                if D[i][j] not in itemAllCount.keys():
                    itemAllCount[D[i][j]] = 0
                if D[i][j] not in itemThisRow.keys():
                    itemAllCount[D[i][j]] = itemAllCount[D[i][j]] + 1# 支持度计数加一
                    itemThisRow[D[i][j]] = True

    #             print(itemAllCount,"---------------------",itemThisRow)

        c_itemList = []
        # 根据minSup筛选候选字符
        for item in itemAllCount.keys():
            if itemAllCount[item]/len(D) >= minSup:
                c_itemList.append(item) 

        # 将筛选后的字符组成新的候选项，加入候选集合
        for c_item in c_itemList:
            newLk.append(Ck+tuple([c_item]))

    #     print("--------------------")
    #     print(returnList)
    #     print(suffixDic)
    #     print("--------------------")
    #     print("\n")
    oldSuf = suffixDic
    newSuffixDic = {}
    for Ck in newLk:
        newSuffixDic[Ck]={}
        tgt = Ck[-1]
        oldSufLocList = oldSuf[Ck[:-1]]#切片倒数第二个，新加的后缀的前一个
        for i in oldSufLocList.keys():
            for j in range(oldSufLocList[i]+1, len(D[i])):
                if D[i][j] == tgt:
                    newSuffixDic[Ck][i] = j
                    break

    return newLk, newSuffixDic

In [98]:
data = [
    ['b', 'e', 'c', 'd', 'p'],
    ['a', 'b', 'c', 'd'],
    ['b', 'c', 'b', 'd', 'e'],
    ['n', 'c', 'b', 'z', 'c', 'e', 'e', 'o', 'h'],
    ['b', 'b', 'b', 'c', 'd', 'g',]
]
D = data
minSup = 0.3
L1, suffixDic = createCInit(D, minSup)
L = [L1]
i = 0
while len(L[i])>0:
    k = i + 2# k就是Lk的k，代表第几次生成的序列
    # 生成新的候选集合
    # 根据新的候选集合获取新的后缀
    Lk,suffixDic = genNewFixAndFixDict(D, L[i], suffixDic,minSup)
    # 加入到集合中
    L.append(Lk)
    i = i + 1
print(L)

[[('b',), ('e',), ('c',), ('d',)], [('b', 'e'), ('b', 'c'), ('b', 'd'), ('b', 'b'), ('c', 'd'), ('c', 'b'), ('c', 'e')], [('b', 'c', 'd'), ('b', 'c', 'e'), ('b', 'b', 'd'), ('c', 'b', 'e')], []]
